# Predicting HDB Resale Prices in Singapore

# Problem Statement

Due to covid and the presence of lockdowns which has affected public housing project delays in Singapore, HDB resale prices has been steadily increasing and has become a common dicussion topics especially amongst first time young buyers.  

I am curious to find out what features influence HDB resale prices and help potential buyers find out if the current asking prices of HDBs are reasonable by using regression models to predict HDB prices.

The increasing cost of living comes to mind for young Singaporeans looking to purchase a home and start a family. This model would serve a guide for them as part of their home purchase decision making process.

Regression models for consideration: Linear, Lasso, Ridge, XGBoost, Neural Networks

Success Metrics: Model performance will be guided by RMSE. We will seek to find the best performing model based on the lowest RMSE score. 

# Data Sources

Data received from https://data.gov.sg/dataset/resale-flat-prices with HDB resale transactions dating back to 1990. Most recent data is Oct 2021.

In [1]:
import pandas as pd

In [3]:
hdb_1990_1999 = pd.read_csv('../datasets/resale-flat-prices-based-on-approval-date-1990-1999.csv')
hdb_2000_2012feb = pd.read_csv('../datasets/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
hdb_2012mar_2014dec = pd.read_csv('../datasets/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
hdb_2015_2016 = pd.read_csv('../datasets/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
hdb_2017_to_recent = pd.read_csv('../datasets/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [7]:
print(hdb_1990_1999.shape, hdb_2000_2012feb.shape, hdb_2012mar_2014dec.shape, hdb_2015_2016.shape, hdb_2017_to_recent.shape)

(287196, 10) (369651, 10) (52203, 10) (37153, 11) (109322, 11)


In [9]:
hdb_1990_1999.shape[0] + hdb_2000_2012feb.shape[0] + hdb_2012mar_2014dec.shape[0] + hdb_2015_2016.shape[0] + hdb_2017_to_recent.shape[0]

855525

In [20]:
# briefly examine df
hdb_2017_to_recent.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [10]:
hdb_2017_to_recent['flat_model'].unique()

array(['Improved', 'New Generation', 'DBSS', 'Standard', 'Apartment',
       'Simplified', 'Model A', 'Premium Apartment', 'Adjoined flat',
       'Model A-Maisonette', 'Maisonette', 'Type S1', 'Type S2',
       'Model A2', 'Terrace', 'Improved-Maisonette', 'Premium Maisonette',
       'Multi Generation', 'Premium Apartment Loft', '2-room'],
      dtype=object)

In [22]:
# checking data types
hdb_2017_to_recent.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object

In [23]:
hdb_2017_to_recent['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

# Feature Engineering

- Create longitude and latitude columns

Real estate prices are closely driven by location. We will create longitude and latitude data from the block and street_name columns to get their exact locations by calling the government's OneMap API. 

- Create price per sqf
- Create age of unit
- Create proximity to nearest MRT station column
- Create proximity to CBD column
- Create proximity to nearest school column
- Create proximity to nearest mall column
- Create proximity to nearest hawker centre column
- Create proximity to nearest supermarket column
- Modifying storey range column into ordinal variables: higher floor should have a higher value
- Modifying resale price to 2021's prices to adjust for inflation

In [13]:
hdb_2017_to_recent['longitude'] = ''
hdb_2017_to_recent['latitude'] = ''

In [14]:
hdb_2017_to_recent.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,longitude,latitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,,
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,,
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,,
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,,
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,,


In [32]:
import requests

In [17]:
url = 'https://developers.onemap.sg/commonapi/search?searchVal=406 ANG MO KIO AVE 10&returnGeom=Y&getAddrDetails=Y'

In [18]:
res = requests.get(url)

In [19]:
data = res.json()

In [20]:
longitude = data['results'][0]['LONGITUDE']
longitude

'103.853879910407'

In [21]:
latitude = data['results'][0]['LATITUDE']
latitude

'1.36200453938712'

In [103]:
# function to get lat and long of the hdb using OneMap API
def get_coordinate(df):
    row = 0
    
    for block, street in zip(df['block'], df['street_name']):
        url = f'https://developers.onemap.sg/commonapi/search?searchVal={block} {street}&returnGeom=Y&getAddrDetails=Y'
        res = requests.get(url)

        # Check Status Code
        if res.status_code != 200:
            print("Error",res.status_code)

        else:

            print(f'Retrieved coordinates from {url}')
            data = res.json()
            
            try:
                longitude = data['results'][0]['LONGITUDE']
                latitude = data['results'][0]['LATITUDE']
            
            # if no data, we will return empty strings
            except:
                longitude = ''
                latitude = ''

            df['longitude'][row] = longitude
            df['latitude'][row] = latitude
            
            row += 1
            
            # to add another line of should save the df with coordinates into a csv

    return df

In [59]:
testing = hdb_2017_to_recent.head()

In [104]:
get_coordinate(testing)

Retrieved coordinates from https://developers.onemap.sg/commonapi/search?searchVal=406 ANG MO KIO AVE 10&returnGeom=Y&getAddrDetails=Y
Retrieved coordinates from https://developers.onemap.sg/commonapi/search?searchVal=108 ANG MO KIO AVE 4&returnGeom=Y&getAddrDetails=Y
Retrieved coordinates from https://developers.onemap.sg/commonapi/search?searchVal=602 ANG MO KIO AVE 5&returnGeom=Y&getAddrDetails=Y


<ipython-input-103-839b91e843cc>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'][row] = longitude
<ipython-input-104-806e2a331d9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  get_coordinate(test)
<ipython-input-103-839b91e843cc>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'][row] = latitude


Retrieved coordinates from https://developers.onemap.sg/commonapi/search?searchVal=465 ANG MO KIO AVE 10&returnGeom=Y&getAddrDetails=Y
Retrieved coordinates from https://developers.onemap.sg/commonapi/search?searchVal=601 ANG MO KIO AVE 5&returnGeom=Y&getAddrDetails=Y


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,longitude,latitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,103.853879910407,1.36200453938712
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,103.837974822369,1.37094273993861
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,103.835368226602,1.38070883044887
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,103.857200967235,1.3662010408294
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,103.835131742647,1.38104135523576


In [ ]:
# how to calculate distance between 2 coordinates
    # https://www.kite.com/python/answers/how-to-find-the-distance-between-two-lat-long-coordinates-in-python

# Potential Challenges

There is quite a number of features to be created, over a large dataset of 109k columns (which is just 2017-2021 data). In total, there are over 850,000 rows for the entire dataset. We will consider to apply our model trainings on a small subset of the dataset first to better handle computational resources.

# Potential EDA ideas

- study pricing trend of HDB units which were transacted multiple times (if any)
- analyze pricing trend of different flat model
- looking at flat model popularity throughout the years
- looking at estate popularity throughout the years